In [1]:
import numpy as np
import pandas as pd
from scraperFunctions import vtiScraper,vxusScraper, bndScraper, bndxScraper
from updaterFunctions import vtiUpdater, vxusUpdater, bndUpdater, bndxUpdater
from helperFunctions import etfPriceData, etfCurrPrices

In [ ]:
# Running scrapers and saving data(If you already have scraped historical data, go to updater)
vtiScraper()
print('VTI done')
vxusScraper()
print('VXUS done')
bndScraper()
print('BND done')
bndxScraper()
print('BNDX done')

In [2]:
# Updating data (If already have updated historical data, go to reader)
vtiUpdater()
vxusUpdater()
bndUpdater()
bndxUpdater()

In development
In development
In development
In development


In [3]:
# Reading saved data (Market cap is in billions)
vti_MC_data = pd.read_csv('vti_MC_Data.csv')
vxus_MC_data = pd.read_csv('vxus_MC_Data.csv')
bnd_MC_data = pd.read_csv('bnd_MC_Data.csv')
bndx_MC_data = pd.read_csv('bndx_MC_Data.csv')

In [4]:
# Getting historical price data for etfs (This only takes a couple seconds to run.)
etfPriceData()
etf_price_data = pd.read_csv('etf_price_data.csv')
etf_price_data.Date = pd.to_datetime(etf_price_data.Date)

In [5]:
df = vti_MC_data.rename(columns={"Market Cap":'VTI'}).merge(vxus_MC_data.rename(columns={"Market Cap":'VXUS'}),on='date',how='outer').merge(bnd_MC_data.rename(columns={"Market Cap":'BND'}),on='date',how='outer').merge(bndx_MC_data.rename(columns={"Market Cap":'BNDX'}),on='date',how='outer').sort_values(by=['date'],ascending=False).reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])

In [6]:
# Getting end of quarter prices and market caps
etfs = ['VTI','VXUS','BND','BNDX']
etfEOQprices = []
for etf in etfs:
    maxDate = etf_price_data[(etf_price_data.Date.dt.year == 2020) & (etf_price_data.Date.dt.month == 3)].Date.max()
    price = etf_price_data[(etf_price_data.Date == maxDate) & (etf_price_data.ETF == etf)].AdjClose.reset_index(drop=True)[0]
    etfEOQprices.append(price)
etfEOQmarketcaps = []
for etf in etfs:
    mc = df[(df.date.dt.year == 2020) & (df.date.dt.month == 3)][['date',etf]].dropna()[etf].reset_index(drop=True)[0]
    etfEOQmarketcaps.append(mc)
# Need to subtract US bonds from World bond market cap
etfEOQmarketcaps[3] = etfEOQmarketcaps[3] - etfEOQmarketcaps[2]
etfRecentPrices = []
for etf in etfs:
    price = etf_price_data[(etf_price_data.Date == etf_price_data.Date.max()) & (etf_price_data.ETF == etf)].AdjClose.reset_index(drop=True)[0]
    etfRecentPrices.append(price)

In [7]:
# Enter in total amount you would like to invest under amountInvested 
amountInvested = 11496

In [8]:
# GENERATING MARKET PORTFOLIO
# format [U.S. Stocks, International Stocks, U.S. Bonds, International Bonds] (in billions for market cap)
EOQmarketcaps = np.array(etfEOQmarketcaps)
EOQprices = np.array(etfEOQprices)
recentPrices = np.asarray(etfRecentPrices)
# Market proportions for each etf on end of quarter
EOQproportions = EOQmarketcaps/sum(EOQmarketcaps)
# Ratio of recent prices and end of quarter prices
priceRatios = recentPrices / EOQprices
# Multiply ratios to market proportions
products = EOQproportions * priceRatios
# Market weights if fractional shares existed
revisedProportions = products / sum(products)
# Amount of money invested in each etf if fractional shares existed
desiredValues = revisedProportions * amountInvested
# Number of shares of etfs to buy if fractional shares existed
currentPrices = np.array(etfCurrPrices(['VTI', 'VXUS', 'BND', 'BNDX']))
shares = desiredValues/currentPrices
# Optimize to get as close to 0 since fractional shares with Schwab doesn't exist
def asdf(a):
    return amountInvested - sum(np.round(shares * a)*currentPrices)
a = .90
while asdf(a) > 0:
    if a <.99999:
        a+=.00001
    else:
        break
    if asdf(a) < 0:
        a-=.00001
        break
# Actual shares of each etf to buy
actualShares = np.round(shares * a)
print('VTI, VXUS, BND, BNDX')
print(actualShares)

VTI, VXUS, BND, BNDX
[26. 62. 33. 40.]


In [9]:
actualProportions = (actualShares * currentPrices)/sum(actualShares * currentPrices)
print('Theoretical proportions')
print(revisedProportions)
print('Actual proportions')
print(actualProportions)

Theoretical proportions
[0.31199662 0.23579429 0.25379039 0.1984187 ]
Actual proportions
[0.31521732 0.23678533 0.25114313 0.19685423]


In [10]:
# Cash left over. You can invest this cash into VTIP (Vanguard's US TIPS ETF) or something else if you'd like.
cash = amountInvested - sum((actualShares * currentPrices))
print(np.round(cash,2))

1.2


In [11]:
# How much it costs to maintain portfolio per year
costs = [.0003,.0008,.00035,.0008]
print(sum((actualShares * currentPrices)*costs))

6.085082


In [12]:
# REBALANCE (run this cell if you already have money invested in the market portfolio)
# Enter in number of shares here in current port in the format of VTI, VXUS, BND, BNDX
currentPort = np.array([12.0588,27.0695,16.0667,21.0345])
trades = actualShares - currentPort
# Amount left for trades
amountInvestable = amountInvested - sum(currentPort * currentPrices)
# Need to make sure trades don't make balance go over amount of money available
def asdf(a):
    return amountInvestable - sum(np.round(trades * a)*currentPrices)
a = .90
while asdf(a) > 0:
    if a <.99999:
        a+=.00001
    else:
        break
    if asdf(a) < 0:
        a-=.00001
        break
actualTrades = np.round(trades * a)
print('Trades: VTI, VXUS, BND, BNDX')
print(actualTrades)
print()
print('Cash left over')
print(asdf(a))

Trades: VTI, VXUS, BND, BNDX
[14. 34. 17. 19.]

Cash left over
26.06800099999964


In [13]:
portAfterTrades = currentPort + actualTrades
print('Theoretical proportions')
print(revisedProportions)
print()
print('Actual proportions after rebalance')
print((portAfterTrades * currentPrices)/sum(portAfterTrades * currentPrices))

Theoretical proportions
[0.31199662 0.23579429 0.25379039 0.1984187 ]

Actual proportions after rebalance
[0.31661516 0.23373731 0.25219634 0.19745118]
